In [ ]:
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

cards = []
card_ids = [x for x in range(2389, 2855)]
# card_ids = [51]
try:
    for card_id in card_ids:
        url = f"https://card-gorilla.com/card/detail/{card_id}"
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        time.sleep(1)

        # 페이지 타입 체크
        if not driver.current_url.endswith(f"/detail/{card_id}"):
            print(f"🚫 Card {card_id}: 존재하지 않거나 메인으로 리다이렉트 됨. 건너뜁니다.")
            continue

        # 신규발급 중단 카드 체크
        try:
            discontinued = driver.find_element(
                By.CSS_SELECTOR,
                "#q-app section div.card_detail.fr-view section div article.card_top div.data_area div.btn_wrap div a span b"
            ).text.strip()
            if discontinued == "신규발급이 중단된 카드입니다.":
                print(f"🚫 Card {card_id}: 신규발급 중단된 카드. 건너뜁니다.")
                continue
        except NoSuchElementException:
            pass  # 태그가 없으면 정상 카드

        # 1) 요약 박스 로드
        try:
            summary_box = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.in_box"))
            )
        except TimeoutException:
            print(f"🚫 Card {card_id}: 요약 박스 로드 실패. 건너뜁니다.")
            continue

        summary_text = [line.strip() for line in summary_box.text.splitlines() if line.strip()]
        if summary_text and summary_text[0] == "비교함 담기":
            summary_text.pop(0)

        # name / brand / fee
        name  = summary_text[0]
        brand = summary_text[1] if len(summary_text) > 1 else ""
        fee   = next((line for line in summary_text if "원" in line), "")

        # 3) 혜택 목록 추출
        try:
            bene_area = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "article.cmd_con.benefit div.lst.bene_area"))
            )
            dt_list = bene_area.find_elements(By.CSS_SELECTOR, "dl dt")
        except TimeoutException:
            print(f"⚠️ Card {card_id}: 혜택 영역 로드 실패, 혜택 없이 저장합니다.")
            dt_list = []

        benefits = []
        for dt in dt_list:
            try:
                category = dt.find_element(By.CSS_SELECTOR, "p.txt1").text.strip()
                summary  = dt.find_element(By.TAG_NAME, "i").text.strip()
                driver.execute_script("arguments[0].click();", dt)
                time.sleep(0.5)
                dd = dt.find_element(By.XPATH, "./following-sibling::dd")
                details = [line.strip() for line in dd.text.split("\n") if line.strip()]
                benefits.append({
                    "category": category,
                    "summary": summary,
                    "details": details
                })
            except NoSuchElementException:
                continue

        cards.append({
            "id": card_id,
            "name": name,
            "brand": brand,
            "fee": fee,
            "summary_box": "\n".join(summary_text),
            "benefits": benefits
        })

    # JSON 파일로 저장
    with open("cards.json", "w", encoding="utf-8") as f:
        json.dump(cards, f, ensure_ascii=False, indent=2)
    print("✅ cards.json 저장 완료")

finally:
    driver.quit()


🚫 Card 2391: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2397: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2403: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2405: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2410: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2411: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2412: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2413: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2414: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2415: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2417: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2424: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2425: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2436: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2438: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2439: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2452: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2455: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2458: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2465: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2483: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2489: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2498: 존재하지 않거나 메인으로 리다이렉트 됨. 건너뜁니다.
🚫 Card 2499: 존재하지 않거나 메인으로 리다이렉트 됨. 건너뜁니다.
🚫 Card 2500: 존재하지 않거나 메인으로 리다이렉트 됨. 건너뜁니다.
🚫 Card 2520: 존재하지 않거나 메인으로 리다이렉트 됨. 건너뜁니다.
🚫 Card 2522: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2523: 신규발급 중단된 카드. 건너뜁니다.
🚫 Card 2525: 신규발급 중단된 카드. 건너뜁니다.
🚫 C